In [1]:
import pandas as pd

In [2]:
armoured_beast = pd.read_pickle('data/stats/armoured_beast_stats_v1.pkl')
armoured_colossal = pd.read_pickle("data/stats/armoured_colossal_stats_v1.pkl")
attack_armoured = pd.read_pickle("data/stats/attack_armoured_stats_v1.pkl")
attack_beast = pd.read_pickle("data/stats/attack_beast_stats_v1.pkl")
attack_colossal = pd.read_pickle("data/stats/attack_colossal_stats_v1.pkl")
beast_colossal = pd.read_pickle("data/stats/beast_colossal_stats_v1.pkl")
cart_armoured = pd.read_pickle("data/stats/cart_armoured_stats_v1.pkl")
cart_attack = pd.read_pickle("data/stats/cart_attack_stats_v1.pkl")
cart_beast = pd.read_pickle("data/stats/cart_beast_stats_v1.pkl")
cart_colossal = pd.read_pickle("data/stats/cart_colossal_stats_v1.pkl")
cart_female = pd.read_pickle("data/stats/cart_female_stats_v1.pkl")
cart_warhammer = pd.read_pickle("data/stats/cart_warhammer_stats_v1.pkl")
female_armoured = pd.read_pickle("data/stats/female_armoured_stats_v1.pkl")
female_attack = pd.read_pickle("data/stats/female_attack_stats_v1.pkl")
female_beast = pd.read_pickle("data/stats/female_beast_stats_v1.pkl")
female_colossal = pd.read_pickle("data/stats/female_colossal_stats_v1.pkl")
female_warhammer = pd.read_pickle("data/stats/female_warhammer_stats_v1.pkl")
jaw_armoured = pd.read_pickle("data/stats/jaw_armoured_stats_v1.pkl")
jaw_attack = pd.read_pickle("data/stats/jaw_attack_stats_v1.pkl")
jaw_beast = pd.read_pickle("data/stats/jaw_beast_stats_v1.pkl")
jaw_cart = pd.read_pickle("data/stats/jaw_cart_stats_v1.pkl")
jaw_colossal = pd.read_pickle("data/stats/jaw_colossal_stats_v1.pkl")
jaw_female = pd.read_pickle("data/stats/jaw_female_stats_v1.pkl")
jaw_warhammer = pd.read_pickle("data/stats/jaw_warhammer_stats_v1.pkl")
warhammer_armoured = pd.read_pickle("data/stats/warhammer_armoured_stats_v1.pkl")
warhammer_attack = pd.read_pickle("data/stats/warhammer_attack_stats_v1.pkl")
warhammer_beast = pd.read_pickle("data/stats/warhammer_beast_stats_v1.pkl")
warhammer_colossal = pd.read_pickle("data/stats/warhammer_colossal_stats_v1.pkl")


In [82]:
jaw_attack

,win_ratio,duel_mean_damage,hit_ratio,mean_turns_to_win
Jaw Titan,18.45,28.6500,51.778700,3.473713
Attack Titan,81.55,42.2128,51.906967,3.054077


In [83]:
jaw_armoured

,win_ratio,duel_mean_damage,hit_ratio,mean_turns_to_win
Jaw Titan,75.44,55.4314,35.771335,7.229984
Armoured Titan,24.56,28.8786,48.836743,8.226384


In [84]:
jaw_colossal

,win_ratio,duel_mean_damage,hit_ratio,mean_turns_to_win
Jaw Titan,61.44,69.3704,70.801474,6.113118
Colossal Titan,38.56,34.5706,58.109035,5.386670


In [85]:
jaw_beast

,win_ratio,duel_mean_damage,hit_ratio,mean_turns_to_win
Jaw Titan,99.91,39.9940,55.113380,3.758783
Beast Titan,0.09,8.2744,46.351384,7.000000


In [86]:
jaw_female

,win_ratio,duel_mean_damage,hit_ratio,mean_turns_to_win
Jaw Titan,66.54,34.6137,47.555941,3.766005
Female Titan,33.46,30.9128,50.460013,4.237597


In [87]:
jaw_warhammer

,win_ratio,duel_mean_damage,hit_ratio,mean_turns_to_win
Jaw Titan,29.28,31.3962,50.021736,4.076844
Warhammer Titan,70.72,39.2790,39.250833,3.191176


In [88]:
jaw_cart

,win_ratio,duel_mean_damage,hit_ratio,mean_turns_to_win
Jaw Titan,99.26,39.9141,55.083001,3.725871
Cart Titan,0.74,11.6193,50.033414,5.756757


In [21]:
d = {
    'attack':[50, attack_armoured['win_ratio'][0], attack_colossal['win_ratio'][0], attack_beast['win_ratio'][0], 
              female_attack['win_ratio'][1], warhammer_attack['win_ratio'][1], cart_attack['win_ratio'][1], jaw_attack['win_ratio'][1]],
    
    'armoured':[attack_armoured['win_ratio'][1], 50, armoured_colossal['win_ratio'][0], armoured_beast['win_ratio'][0],
               female_armoured['win_ratio'][1], warhammer_armoured['win_ratio'][1], cart_armoured['win_ratio'][1], jaw_armoured['win_ratio'][1]],
    
    'colossal':[attack_colossal['win_ratio'][1], armoured_colossal['win_ratio'][1], 50, beast_colossal['win_ratio'][1], 
                female_colossal['win_ratio'][1], warhammer_colossal['win_ratio'][1], cart_colossal['win_ratio'][1], jaw_colossal['win_ratio'][1]],
    
    'beast':[attack_beast['win_ratio'][1], armoured_beast['win_ratio'][1], beast_colossal['win_ratio'][0], 50, female_beast['win_ratio'][1],
            warhammer_beast['win_ratio'][1], cart_beast['win_ratio'][1], jaw_beast['win_ratio'][1]],
    
    'female':[female_attack['win_ratio'][0], female_armoured['win_ratio'][0], female_colossal['win_ratio'][0], female_beast['win_ratio'][0],
             50, female_warhammer['win_ratio'][0], cart_female['win_ratio'][1], jaw_female['win_ratio'][1]],
    
    'warhammer':[warhammer_attack['win_ratio'][0], warhammer_armoured['win_ratio'][0], warhammer_colossal['win_ratio'][0], warhammer_beast['win_ratio'][0],
                female_warhammer['win_ratio'][1], 50, cart_warhammer['win_ratio'][1], jaw_warhammer['win_ratio'][1]],
    
    'cart':[cart_attack['win_ratio'][0], cart_armoured['win_ratio'][0], cart_colossal['win_ratio'][0], cart_beast['win_ratio'][0], cart_female['win_ratio'][0],
           cart_warhammer['win_ratio'][0], 50, jaw_cart['win_ratio'][1]],
    
    'jaw':[jaw_attack['win_ratio'][0], jaw_armoured['win_ratio'][0], jaw_colossal['win_ratio'][0], jaw_beast['win_ratio'][0], jaw_female['win_ratio'][0], 
          jaw_warhammer['win_ratio'][0], jaw_cart['win_ratio'][0], 50]
    }

In [26]:
win_ratio_matrix = pd.DataFrame(data=d, index=['attack', 'armoured', 'colossal', 'beast', 'female', 'warhammer', 'cart', 'jaw'])
heatmap = win_ratio_matrix.T.style.background_gradient(axis=1)
heatmap

,attack,armoured,colossal,beast,female,warhammer,cart,jaw
attack,50.000000,97.600000,93.380000,100.000000,92.050000,61.510000,99.980000,81.550000
armoured,2.400000,50.000000,21.180000,99.780000,39.490000,11.240000,97.360000,24.560000
colossal,6.620000,78.820000,50.000000,99.980000,61.310000,9.150000,99.800000,38.560000
beast,0.000000,0.220000,0.020000,50.000000,0.360000,0.020000,25.900000,0.090000
female,7.950000,60.510000,38.690000,99.640000,50.000000,15.690000,97.430000,33.460000
warhammer,38.490000,88.760000,90.850000,99.980000,84.310000,50.000000,99.870000,70.720000
cart,0.020000,2.640000,0.200000,74.100000,2.570000,0.130000,50.000000,0.740000
jaw,18.450000,75.440000,61.440000,99.910000,66.540000,29.280000,99.260000,50.000000


In [25]:
# Salva la heatmap in un file html

html_str = heatmap.render()
Html_file= open("win_rate_heatmap.html","w")
Html_file.write(html_str)
Html_file.close()